In [1]:
#importing of libraries
#data manipulation
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from fuzzywuzzy import process
#nltk.download('punkt_tab')/
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
df = pd.read_csv('data.csv', chunksize=100000)

In [3]:
final_df = pd.DataFrame()
for chunk in df:
    columns = ['question_id', 'question_user_id', 'question_language',
               'question_content', 'question_topic', 'question_sent','question_user_type',
               'question_user_status', 'question_user_country_code']
    filtered = chunk[(chunk['question_language'] =='eng') &
                     (chunk['response_language'] =='eng')]
    output = filtered[columns]
    final_df = pd.concat([final_df, output], ignore_index=True)
    

C:\Users\sehal\AppData\Local\Temp\ipykernel_44556\1694429600.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in df:
C:\Users\sehal\AppData\Local\Temp\ipykernel_44556\1694429600.py:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in df:


In [4]:
final_df.head()

,question_id,question_user_id,question_language,question_content,question_topic,question_sent,question_user_type,question_user_status,question_user_country_code
0,3849061,521327,eng,Q this goes to wefarm. is it possible to get f...,NaN,2017-11-22 12:25:05+00,farmer,live,ug
1,3849084,6642,eng,Q-i have stock rabbit's urine for 5 weeks mash...,rabbit,2017-11-22 12:25:10+00,farmer,destroyed,ke
2,3849098,526375,eng,Q J Have Mi 10000 Can J Start Aproject Of Pout...,poultry,2017-11-22 12:25:12+00,farmer,zombie,ug
3,3849100,237506,eng,WHERE DO I GET SEEDS OF COCONUT?,pig,2017-11-22 12:25:12+00,farmer,destroyed,ke
4,3849100,237506,eng,WHERE DO I GET SEEDS OF COCONUT?,coconut,2017-11-22 12:25:12+00,farmer,destroyed,ke


In [41]:
final_df.shape

(11523993, 10)

In [6]:
#Data exploration (missing data)
final_df.isnull().sum()

question_id                         0
question_user_id                    0
question_language                   0
question_content                    0
question_topic                1548673
question_sent                       0
question_user_type                  0
question_user_status                0
question_user_country_code          0
dtype: int64

In [7]:
#Data conversion (date column to datetime)
final_df['question_sent'] = pd.to_datetime(final_df['question_sent'], format='ISO8601')

In [8]:
final_df['month'] = final_df['question_sent'].dt.month

In [9]:
#function to deduce what season question encompasses
farming_season = ['planting', 'growing', 'harvesting']
def season_matching(data, action, threshold=85):
    stop_words = set(stopwords.words('english'))
    lemma = WordNetLemmatizer()
    if not isinstance(data, str):
        return None
    words = word_tokenize(data.lower())
    lemma_words = [lemma.lemmatize(word, pos='v') for word in words if word.isalnum() and word not in stop_words]
    
    
    matched= set()
    for token in lemma_words:
        match, score = process.extractOne(token, action)
        if score>=threshold:
            matched.add(match)
        
    if not matched:
        matched.add('General')
    
    return ','.join(list(matched))

In [10]:
#checking for the number of unique questions actions
final_df['question_id'].nunique()

2913361

In [11]:
#Dataframe to collate only unique questions based on question id's
unique_df = final_df.drop_duplicates(subset='question_id',keep='first')

In [12]:
# Data Frame of unique ID's split into two halves for easier analysis
first = unique_df[:1456680]
second = unique_df[1456680:]

In [13]:
#applying season function to the unique question ID's
first['season'] = first['question_content'].apply(lambda x: season_matching(x, farming_season))

C:\Users\sehal\AppData\Local\Temp\ipykernel_44556\1668448038.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first['season'] = first['question_content'].apply(lambda x: season_matching(x, farming_season))


In [14]:
#checking results of function application
first['season'].value_counts()

season
General                        1207373
planting                        179700
growing                          59516
planting,growing                  6507
harvesting                        2800
planting,harvesting                611
harvesting,growing                 150
planting,harvesting,growing         23
Name: count, dtype: int64

In [15]:
#applying season function to the unique question ID's
second['season'] = second['question_content'].apply(lambda x: season_matching(x, farming_season))

C:\Users\sehal\AppData\Local\Temp\ipykernel_44556\1623846745.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second['season'] = second['question_content'].apply(lambda x: season_matching(x, farming_season))


In [16]:
unique = pd.concat([first, second])

In [17]:
#creating a map of unique question ID's and Expected seasons for further application
unique_map = unique[['question_id', 'season']]

In [18]:
unique_map['season'].value_counts()

season
General                        2409157
planting                        360753
growing                         122824
planting,growing                 13515
harvesting                        5667
planting,harvesting               1142
harvesting,growing                 264
planting,harvesting,growing         39
Name: count, dtype: int64

In [19]:
#checking proportion of data by Country/region
final_df['question_user_country_code'].value_counts()/final_df.shape[0]*100

question_user_country_code
ke    61.790874
ug    38.206679
gb     0.002360
tz     0.000087
Name: count, dtype: float64

In [20]:
#splitting data based on region 
df_ke = final_df[final_df['question_user_country_code']=='ke'].copy()
df_ug = final_df[final_df['question_user_country_code']=='ug'].copy()
df_gb = final_df[final_df['question_user_country_code']=='gb'].copy()
df_tz = final_df[final_df['question_user_country_code']=='tz'].copy()    

In [21]:
#mapping the identified seasons to the data on question ID's
df_ke = pd.merge(df_ke, unique_map, on='question_id', how='inner')
df_ug = pd.merge(df_ug, unique_map, on='question_id', how='inner')
df_gb = pd.merge(df_gb, unique_map, on='question_id', how='inner')
df_tz = pd.merge(df_tz, unique_map, on='question_id', how='inner')

In [22]:
#define Kenya's seasons
def general_season(month):
    if month in [3,4,5,10,11,12]:
        a = 'planting,growing'
        return a.split(',')
    elif month in [1,2,6,7,8,]:
        return ['harvesting']
    else:
        return ['General']
    

In [23]:
ke_analysis = df_ke.copy()

In [24]:
ke_analysis['General_season'] = ke_analysis['month'].apply(lambda x: general_season(x))

In [25]:
ke_analysis['General_season'].value_counts()

General_season
[planting, growing]    3991107
[harvesting]           2363500
[General]               766169
Name: count, dtype: int64

In [26]:
#using the MultiLabelBinarizer to create a Bianary table of for the observed data
mlb = MultiLabelBinarizer()
observed =pd.DataFrame((mlb.fit_transform(ke_analysis['General_season'])), columns=mlb.classes_)

In [27]:
observed

,General,growing,harvesting,planting
0,0,1,0,1
1,0,1,0,1
2,0,1,0,1
3,0,1,0,1
4,0,1,0,1
...,...,...,...,...
7120771,0,1,0,1
7120772,0,1,0,1
7120773,0,0,1,0
7120774,0,0,1,0


In [28]:
ke_analysis['season'] = ke_analysis['season'].apply(lambda x: x.split(','))

In [29]:
#using the MultiLabelBinarizer to create a Binary table for the expected data inferred from the questions
expected =pd.DataFrame((mlb.fit_transform(ke_analysis['season'])), columns=mlb.classes_)

In [30]:
#performing a chisquared test of independence to determine if questions are associated with Kenya's farming season
from scipy.stats import chi2_contingency

for action in mlb.classes_:
    table = pd.crosstab(observed[action], expected[action])
    chi2, p, dof, ex = chi2_contingency(table)
    print(f"{action}: chi2={chi2:.2f}, p={p:.3f}")

General: chi2=3160.38, p=0.000
growing: chi2=174.00, p=0.000
harvesting: chi2=20.38, p=0.000
planting: chi2=3102.89, p=0.000


In [31]:
#defining Uganda's seasons
def general_season_ug(month):
    if month in [3,4,5,9,10,11]:
        a = 'planting,growing'
        return a.split(',')
    elif month in [6,7,8,12,1,2]:
        return ['harvesting']
    else:
        return ['General']

In [32]:
df_ug['Observed Season']=df_ug['month'].apply(lambda x: general_season_ug(x)) 

In [33]:
df_ug['Observed Season'].value_counts()

Observed Season
[planting, growing]    2303517
[harvesting]           2099418
Name: count, dtype: int64

In [34]:
ug_analysis = df_ug[df_ug['season']!='General']

In [35]:
ug_analysis['season'] = ug_analysis['season'].apply(lambda x: x.split(','))

C:\Users\sehal\AppData\Local\Temp\ipykernel_44556\857749182.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ug_analysis['season'] = ug_analysis['season'].apply(lambda x: x.split(','))


In [36]:
ug_analysis['season'].value_counts()

season
[planting]                         812907
[growing]                          273468
[planting, growing]                 43755
[harvesting]                         8118
[planting, harvesting]               2230
[harvesting, growing]                 588
[planting, harvesting, growing]       159
Name: count, dtype: int64

In [37]:
#using the MultiLabelBinarizer to create a Binary table for the expected data inferred from the questions
mlb2 = MultiLabelBinarizer()
ug_observed =pd.DataFrame((mlb2.fit_transform(ug_analysis['Observed Season'])), columns=mlb2.classes_)
ug_expected =pd.DataFrame((mlb2.fit_transform(ug_analysis['season'])), columns=mlb2.classes_)

In [38]:
for action in mlb2.classes_:
    table_ug = pd.crosstab(ug_observed[action], ug_expected[action])
    chi2_ug, p_ug, dof_ug, ex_ug = chi2_contingency(table_ug)
    print(f"{action}: chi2={chi2_ug:.2f}, p={p_ug:.3f}")

growing: chi2=48.32, p=0.000
harvesting: chi2=0.21, p=0.645
planting: chi2=86.90, p=0.000


In [39]:
#creating a Dataframe for analysis in power BI
ke_analysis_for_vis = ke_analysis[['question_id','question_sent','question_user_type','season','General_season']]

In [40]:
ug_analysis_for_vis = df_ug[['question_id','question_sent','question_topic','season','Observed Season']]

In [46]:
print(process.__version__)

AttributeError: module 'fuzzywuzzy.process' has no attribute '__version__'

In [ ]:
ke_analysis_for_vis.to_csv('kenya.csv')

In [ ]:
ug_analysis_for_vis.to_csv('uganda.csv')